# 📥 Fonte → RAW | Catasto Rifiuti ISPRA

Questo notebook gestisce la fase **Fonte → RAW** della pipeline DataCivicLab.

## 🎯 Obiettivo
Scaricare i file **originali (CSV)** dal Catasto Rifiuti ISPRA e salvarli su **Google Drive** come **RAW**, senza alcuna trasformazione.

## 📦 Cosa fa
- scarica i CSV per ciascun anno (es. 2022, 2023)
- salva i file in `data/raw/<fonte>/<dataset>/<anno>/`
- salva i metadati di download in `metadata.json`

## 🚫 Cosa NON fa
- non rinomina colonne
- non pulisce dati
- non converte formati
- non unisce file

## 🧱 Output
- RAW immutabile su Drive
- tracciabilità completa della fonte
- base stabile per la fase successiva (**RAW → CLEAN**)

## 📚 Fonte
ISPRA – Catasto Rifiuti Comunali  
https://www.catasto-rifiuti.isprambiente.it/

Notebook Colab: https://colab.research.google.com/drive/1_siDekvMo8O5-XiBYWgdlZJf-Yce9ZDm?usp=sharing

Output Drive: https://drive.google.com/drive/folders/1vmI6YXFG_ayTkgt8oQ22nUZx4q3OKYQM?usp=drive_link

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import requests
from pathlib import Path
import json
from datetime import datetime


ModuleNotFoundError: No module named 'google'

In [ ]:
BASE_DIR = Path("/content/drive/MyDrive/dataciviclab/data/raw/ispra_catasto_rifiuti")

URL_TEMPLATE = (
    "https://www.catasto-rifiuti.isprambiente.it/"
    "get/getDettaglioComunale.csv.php?&aa={year}"
)

ANNI = list(range(2019, 2024))  # ← scegli anni


In [ ]:
def download_year(year: int):
    year_dir = BASE_DIR / str(year)
    year_dir.mkdir(parents=True, exist_ok=True)

    url = URL_TEMPLATE.format(year=year)
    out_file = year_dir / f"dettaglio_comunale_{year}.csv"

    r = requests.get(url)
    r.raise_for_status()

    out_file.write_bytes(r.content)
    print(f"✅ Scaricato {year}")


In [ ]:
for year in ANNI:
    try:
        download_year(year)
    except Exception as e:
        print(f"❌ Errore {year}: {e}")


✅ Scaricato 2019
✅ Scaricato 2020
✅ Scaricato 2021
✅ Scaricato 2022
✅ Scaricato 2023


In [ ]:
metadata = {
    "dataset": "ISPRA – Catasto Rifiuti – Dettaglio Comunale",
    "source": "https://www.catasto-rifiuti.isprambiente.it/",
    "years": ANNI,
    "downloaded_at": datetime.now().isoformat(),
    "files": {
        str(y): f"{y}/dettaglio_comunale_{y}.csv"
        for y in ANNI
    }
}

with open(BASE_DIR / "metadata.json", "w") as f:
    json.dump(metadata, f, indent=2)
